<a href="https://colab.research.google.com/github/Nithinan-Srikongphan/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [22]:
import pandas as pd
import numpy as np

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import os
os.chdir('/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm')

!pwd # เช็ค path ที่กำลังทำงานอยู่

/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm


#1 **เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)**

**1.1 Data House**

In [25]:
data_house = pd.read_excel("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/Data House.xlsx")
# ตรวจสอบข้อมูลบางส่วน
data_house.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# ตรวจสอบข้อมูลเบื้องต้น
data_house.head()  # ดูแค่บางแถวแรก
data_house.info()  # ดูข้อมูลประเภทของคอลัมน์ และจำนวนข้อมูลในแต่ละคอลัมน์

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1656 non-null   object 
 1   name_th             1655 non-null   object 
 2   name_en             1651 non-null   object 
 3   project_name        539 non-null    object 
 4   company             348 non-null    object 
 5   province            1656 non-null   object 
 6   district            1656 non-null   object 
 7   sub_district        1516 non-null   object 
 8   property_type       1656 non-null   object 
 9   home improvement    1457 non-null   object 
 10  renovate            1445 non-null   object 
 11  public utility      18 non-null     object 
 12  land type           13 non-null     object 
 13  land wide(m)        56 non-null     float64
 14  land long(m)        47 non-null     float64
 15  topsoil             19 non-null     object 
 16  corner

In [27]:
# เลือกคอลัมน์ที่จำเป็น
required_columns = [
    'renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va', 'number_of_story',
    'number_of_bedroom', 'number_of_bathroom', 'direction', 'furniture', 'usable_area',
    'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone'
]

# เลือกเฉพาะคอลัมน์ที่จำเป็น
data_house = data_house[required_columns]

# ตรวจสอบข้อมูลที่เลือก
data_house.head()

,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,no,0,1,45.0,2.0,2.0,2,ไม่ระบุ,ไม่มี,244.0,2.0,390000.0,ชนบท,ศรีบุญเรือง,NaN
1,no,0,0,88.0,1.0,1.0,1,ไม่ระบุ,ไม่มี,164.0,3.0,820000.0,น้ำพอง,ทรายมูล,NaN
2,no,1,0,8.0,1.0,11.0,11,ไม่ระบุ,ไม่ระบุ,489.0,9.0,3260000.0,มัญจาคีรี,กุดเค้า,NaN
3,no,0,0,61.6,2.0,3.0,3,ไม่ระบุ,ไม่ระบุ,203.0,2.0,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,no,0,0,72.3,2.0,3.0,2,ไม่ระบุ,ไม่ระบุ,164.0,2.0,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน


In [80]:
# กลุ่ม zone ที่มีราคาขายเฉลี่ยสูงสุดเรียงลำดับจากมากไปน้อย
average_sell_price_by_zone = data_house.groupby('zone')['sell_price'].mean().sort_values(ascending=False)
average_sell_price_by_zone

,sell_price
zone,
ในเมือง-รื่นรมย์-เทพารักษ์,7.201667e+06
โนนม่วง-มข.-เลี่ยงเมือง-โนนเรือง,5.716039e+06
ทุ่งสร้าง-ศรีจันทร์-ราชมงคล,5.351464e+06
ม.ภาค-บ้านกอก-บึงหนองโคตร,4.605049e+06
บึงเนียม,4.587833e+06
มิตรภาพ-บ้านสำราญ,4.516250e+06
กลางเมือง-เมืองเก่า-กุดกว้าง,4.457461e+06
สามเหลี่ยม-กังสดาล-ม.ขอนแก่น,4.285476e+06
เหล่านาดี-บ้านสะอาด-บ้านโจด,4.250000e+06


**1.2 "7-eleven.json" และ "school.json"**

In [28]:
import json

# อ่านข้อมูลจากไฟล์ JSON
with open("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/7-eleven.json", "r") as f:
    seven_eleven_data = json.load(f)

with open("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/school.json", "r") as f:
    school_data = json.load(f)

**ใช้ไลบรารี geopy เพื่อคำนวณระยะทางระหว่างบ้านและสถานที่**

In [31]:
from geopy.distance import geodesic

# ฟังก์ชันคำนวณระยะทางระหว่าง 2 จุด
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

# ฟังก์ชันตรวจสอบระยะทางจากบ้านไปยัง 7-eleven และโรงเรียน
def check_proximity_to_7_11_and_school(house_lat, house_lon, seven_eleven_data, school_data):
    # ตรวจสอบระยะทางจากบ้านไปยัง 7-eleven
    d_7_11 = any(calculate_distance(house_lat, house_lon, loc['lat'], loc['lon']) <= 1000 for loc in seven_eleven_data)

    # ตรวจสอบระยะทางจากบ้านไปยังโรงเรียน
    d_school = any(calculate_distance(house_lat, house_lon, loc['lat'], loc['lon']) <= 1000 for loc in school_data)

    return d_7_11, d_school

**เพิ่มข้อมูล** d1000-7/11 และ d1000-school ลงใน data_house
ในขั้นตอนนี้เราจะเพิ่มคอลัมน์ d1000-7/11 และ d1000-school ให้กับข้อมูลบ้านเพื่อบ่งบอกว่า 7-11 หรือโรงเรียนอยู่ในรัศมี 1 กิโลเมตรหรือไม่

In [37]:
# ตรวจสอบชื่อคอลัมน์ทั้งหมดใน data_house
print(data_house.columns)

Index(['renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
       'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
       'direction', 'furniture', 'usable_area', 'number_of_parking',
       'sell_price', 'district', 'sub_district', 'zone', 'latitude',
       'longitude'],
      dtype='object')


In [103]:
import json

# อ่านข้อมูลจากไฟล์ JSON
with open("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/7-eleven.json", "r") as f:
    seven_eleven_data = json.load(f)

with open("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/school.json", "r") as f:
    school_data = json.load(f)

# ฟังก์ชันคำนวณระยะทางระหว่าง 2 จุด
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

# ฟังก์ชันตรวจสอบระยะทางจากบ้านไปยัง 7-eleven และโรงเรียน
def check_proximity_to_7_11_and_school(house_lat, house_lon, seven_eleven_data, school_data):
    # ตรวจสอบระยะทางจากบ้านไปยัง 7-eleven
    d_7_11 = any(
        calculate_distance(house_lat, house_lon, float(loc['lat']), float(loc['lon'])) <= 1000
        for loc_list in seven_eleven_data.values() for loc in loc_list # loc is now a dictionary, access lat and lon directly
    )

    # ตรวจสอบระยะทางจากบ้านไปยังโรงเรียน
    d_school = any(calculate_distance(house_lat, house_lon, loc['lat'], loc['lon']) <= 1000 for loc in school_data.values())

    return d_7_11, d_school

In [104]:
def get_distances(house_id, seven_eleven_data, school_data):

    # Get house latitude and longitude (Assuming 'house_id' is the index)
    try:
        house_lat = data_house.loc[house_id, 'latitude']
        house_lon = data_house.loc[house_id, 'longitude']
    except KeyError:
        print(f"House ID {house_id} not found in the dataset.")
        return [], []  # Return empty lists if house not found

    seven_eleven_distances = []
    for loc_list in seven_eleven_data.values():  # Access the values directly
        for loc in loc_list:  # Iterate through the list of dictionaries, loc is a dictionary now
            if loc and 'lat' in loc and 'lon' in loc:  # Check if loc is not empty and contains 'lat' and 'lon'
                distance = calculate_distance(house_lat, house_lon, loc['lat'], loc['lon'])
                seven_eleven_distances.append(distance)

    school_distances = []
    for loc_list in school_data.values():
        for loc in loc_list:
            if loc and 'lat' in loc and 'lon' in loc:  # Check if loc is not empty and contains 'lat' and 'lon'
                distance = calculate_distance(house_lat, house_lon, loc['lat'], loc['lon'])
                school_distances.append(distance)

    return seven_eleven_distances, school_distances

In [105]:
# prompt: แสดงผลเกี่ยวกับระยะทางจากบ้านไปยัง 7-eleven และโรงเรียน

# Assuming 'data_house' DataFrame and 'seven_eleven_data', 'school_data' are already loaded as in the provided code.
# Also assuming 'house_lat' and 'house_lon' are defined for each house.  You'll need to integrate this with your actual data.

def display_distances(house_id, seven_eleven_data, school_data):
    """Displays the distances from a house to nearby 7-Elevens and schools."""

    # Assuming 'house_id' is the index of your DataFrame
    try:
      house_lat = data_house.loc[house_id, 'latitude']  # Accessing by index
      house_lon = data_house.loc[house_id, 'longitude'] # Accessing by index
    except KeyError:
      print(f"House ID {house_id} not found in the dataset.")
      return

    seven_eleven_distances, school_distances = get_distances(house_id, seven_eleven_data, school_data)

    print(f"Distances for house ID: {house_id}")
    if seven_eleven_distances:
        print("Distances to 7-Eleven (meters):", seven_eleven_distances)
    else:
      print("No 7-Elevens found within the specified radius.")
    if school_distances:
        print("Distances to Schools (meters):", school_distances)
    else:
      print("No Schools found within the specified radius.")

# Example usage (replace with your actual house IDs):
# Assuming 'house_id' is the index of your DataFrame, get all unique index values
house_ids_to_check = data_house.index.unique()
for house_id in house_ids_to_check:
    display_distances(house_id, seven_eleven_data, school_data)

Distances for house ID: 0
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 1
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 2
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 3
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 4
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 5
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 6
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 7
No 7-Elevens found within the specified radius.
No Schools found within the specified radius.
Distances for house ID: 8
No 7-Elevens f

**แสดงข้อมูลราคาขายจากมากไปน้อย ในรัศมี 1 กม. **

In [110]:
# Assuming data_house DataFrame is already created and populated as in the previous code.
# Also, assuming 'd1000-7/11' column is added to the data_house dataframe
# and contains the count of 7-11s within 1km radius.

def houses_with_711_in_radius(df):
    """
    Finds houses with the maximum number of 7-11s within a 1km radius and displays their prices.

    Args:
        df: The pandas DataFrame containing house data.  Must have 'sell_price' and 'd1000-7/11' columns.
    Returns:
        A pandas DataFrame showing the houses with the highest number of 7-11s and their prices.
    """
    # Find the maximum number of 7-11s within the 1km radius
    max_711s = df['d1000-7/11'].max()

    # Filter houses with the maximum number of 7-11s
    houses_with_max_711s = df[df['d1000-7/11'] == max_711s]

    # Create a DataFrame with 'sell_price' and 'd1000-7/11'
    result_df = houses_with_max_711s[['sell_price', 'd1000-7/11']]

    return result_df

# Call the function and display the result
result_dataframe = houses_with_711_in_radius(data_house)
result_dataframe

,sell_price,d1000-7/11
0,390000.0,0
1,820000.0,0
2,3260000.0,0
3,5000000.0,0
4,3500000.0,0
...,...,...
1651,437400.0,0
1652,714600.0,0
1653,451000.0,0
1654,910000.0,0


**1.3 ไฟล์ "คำตอบนักประเมิน(ID)"**

In [51]:
# อ่านข้อมูลจากไฟล์ "คำตอบนักประเมิน(ID).xlsx"
estimator_data = pd.read_excel("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/คำตอบนักประเมิน(ID).xlsx")

# ตรวจสอบข้อมูลเบื้องต้น
estimator_data.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


**จัดการ คอลัมน์ "ให้คะแนนทรัพย์" เป็นค่าตัวเลข และเปลี่ยนชื่อเป็น score**

In [157]:
# อ่านข้อมูลจากไฟล์ "คำตอบนักประเมิน(ID).xlsx"
estimator_data = pd.read_excel("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/คำตอบนักประเมิน(ID).xlsx")

# ตรวจสอบข้อมูลเบื้องต้น
estimator_data.head()
# Function to extract the first numerical score from the 'ให้คะแนนทรัพย์' column
def extract_score(score_string):
    if pd.isna(score_string):  # Handle missing values
        return np.nan

    for part in score_string.split():
        try:
            score = int(part)
            if 1 <= score <= 5:
                return score
        except ValueError:
            continue  # Ignore non-numeric parts
    return np.nan  # Return NaN if no valid score is found


# Apply the function to create a new column 'score' with cleaned scores
estimator_data['score'] = estimator_data['ให้คะแนนทรัพย์'].apply(extract_score)

# Print the updated dataframe to check the results
print(estimator_data.head())

   ID ผู้ประเมิน                              post id   ID property_type  \
0              1   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
1              2   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
2              1   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
3              2   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
4              1  https://nayoo.co/khonkaen/posts/495  495    บ้านเดี่ยว   

  ความมั่นใจในการเมินราคาทรัพย์  ราคาทรัพย์  ประเมินราคาซื้อ-ขาย  \
0                        มั่นใจ   2890000.0            2500000.0   
1                        มั่นใจ   2890000.0            2700000.0   
2                        มั่นใจ   2600000.0            2200000.0   
3                        มั่นใจ   2600000.0            2400000.0   
4                        มั่นใจ   6050000.0            5000000.0   

                      ปัจจัยในการประเมินราคาซื้อ-ขาย  \
0                                       5,000-15,000   
1  โครงการนี้ราคาเสนอข

**ใช้คำสั่ง merged เพื่อรวมคอลัมน์ข้อมูลในไฟล์บ้านและนักประเมิน**

In [160]:
import pandas as pd
import numpy as np

# อ่านข้อมูลจากไฟล์ "คำตอบนักประเมิน(ID).xlsx"
estimator_data = pd.read_excel("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/คำตอบนักประเมิน(ID).xlsx")

# อ่านข้อมูลจากไฟล์ "Data House.xlsx"
house_data = pd.read_excel("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/Data House.xlsx")

# ตรวจสอบข้อมูลเบื้องต้น
print("ข้อมูลจากไฟล์ 'คำตอบนักประเมิน(ID).xlsx':")
print(estimator_data.head())
print("ข้อมูลจากไฟล์ 'Data House.xlsx':")
print(house_data.head())

# สร้างฟังก์ชันเพื่อดึงคะแนนที่เป็นตัวเลขด้านหน้าสุดจากคอลัมน์ 'ให้คะแนนทรัพย์'
def extract_score(score_string):
    if pd.isna(score_string):  # ตรวจสอบค่าว่าง
        return np.nan

    for part in score_string.split():
        try:
            score = int(part)
            if 1 <= score <= 5:
                return score
        except ValueError:
            continue  # ข้ามส่วนที่ไม่ใช่ตัวเลข
    return np.nan  # หากไม่พบตัวเลขที่เป็นคะแนน

# เพิ่มคอลัมน์ 'score' โดยใช้ฟังก์ชัน extract_score
estimator_data['score'] = estimator_data['ให้คะแนนทรัพย์'].apply(extract_score)

# เปลี่ยนชื่อคอลัมน์ 'id' ใน house_data เป็น 'ID' เพื่อให้ตรงกับ estimator_data
house_data = house_data.rename(columns={'id': 'ID'}) # This line is added to rename 'id' to 'ID'

# รวมข้อมูลจากทั้งสองไฟล์ผ่านคอลัมน์ 'ID'
merged_data = pd.merge(house_data, estimator_data, on="ID", how="inner")

# แสดงผลลัพธ์หลังการรวม
print("ข้อมูลที่รวมกัน:")
print(merged_data.head())

# บันทึกข้อมูลที่รวมกันลงไฟล์ใหม่
merged_data.to_excel("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/Merged_House_Estimator_Data.xlsx", index=False)
print("บันทึกข้อมูลที่รวมแล้วในไฟล์ 'Merged_House_Estimator_Data.xlsx'")

ข้อมูลจากไฟล์ 'คำตอบนักประเมิน(ID).xlsx':
   ID ผู้ประเมิน                              post id   ID property_type  \
0              1   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
1              2   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
2              1   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
3              2   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
4              1  https://nayoo.co/khonkaen/posts/495  495    บ้านเดี่ยว   

  ความมั่นใจในการเมินราคาทรัพย์  ราคาทรัพย์  ประเมินราคาซื้อ-ขาย  \
0                        มั่นใจ   2890000.0            2500000.0   
1                        มั่นใจ   2890000.0            2700000.0   
2                        มั่นใจ   2600000.0            2200000.0   
3                        มั่นใจ   2600000.0            2400000.0   
4                        มั่นใจ   6050000.0            5000000.0   

                      ปัจจัยในการประเมินราคาซื้อ-ขาย  \
0                                   

In [161]:
# prompt: แสดง score รวมในแต่ละ sub_district

# Assuming 'merged_data' DataFrame is already created and populated as in the previous code.

average_score_by_subdistrict = merged_data.groupby('sub_district')['score'].mean().sort_values(ascending=False)
average_score_by_subdistrict

,score
sub_district,
ศรีบุญเรือง,4.00
บ้านหัน,3.50
หนองแดง,3.50
สงเปือย,3.50
ป่ามะนาว,3.50
...,...
หนองตูม,2.50
แวงน้อย,2.50
คอนฉิม,2.50


In [158]:
print(estimator_data.columns)  # ดูชื่อคอลัมน์ทั้งหมด
print(estimator_data.info())  # ตรวจสอบประเภทข้อมูล และ missing values

Index(['ID ผู้ประเมิน', 'post id', 'ID', 'property_type',
       'ความมั่นใจในการเมินราคาทรัพย์', 'ราคาทรัพย์', 'ประเมินราคาซื้อ-ขาย',
       'ปัจจัยในการประเมินราคาซื้อ-ขาย', 'เหตุผลไม่มั่นใจราคาประเมิน',
       'ให้คะแนนทรัพย์', 'score'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1917 entries, 0 to 1916
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID ผู้ประเมิน                   1917 non-null   int64  
 1   post id                         1917 non-null   object 
 2   ID                              1917 non-null   object 
 3   property_type                   1917 non-null   object 
 4   ความมั่นใจในการเมินราคาทรัพย์   1917 non-null   object 
 5   ราคาทรัพย์                      1916 non-null   float64
 6   ประเมินราคาซื้อ-ขาย             1784 non-null   float64
 7   ปัจจัยในการประเมินราคาซื้อ-ขาย  1786 non-null   object 
 8   เหตุผลไม่มั่นใจ

In [159]:
# เลือกเฉพาะข้อมูลจากนักประเมิน ID=1
estimator_data = estimator_data[estimator_data['ID'] == 1] # เปลี่ยน '01' เป็น 1  เนื่องจากคอลัมน์ ID เป็นตัวเลข

# ตรวจสอบข้อมูล
estimator_data.head()

# ... (Rest of your code)

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์,score
74,1,https://nayoo.co/khonkaen/posts/1,1,บ้านเดี่ยว,มั่นใจ,390000.0,600000.0,"3,000-30,000 บาท/ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0
75,2,https://nayoo.co/khonkaen/posts/1,1,บ้านเดี่ยว,มั่นใจ,390000.0,1300000.0,"ราคาเสนอขาย 1,000 - 3,000 บาท / ตารางวา",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน,4.0


In [74]:
# อ่านข้อมูลจากไฟล์ "คำตอบนักประเมิน(ID).xlsx"
estimator_data = pd.read_excel("/content/drive/MyDrive/Data_BSC_DPDM24/MIDterm/คำตอบนักประเมิน(ID).xlsx")

# ตรวจสอบข้อมูลเบื้องต้น
estimator_data.head()

# เลือกเฉพาะข้อมูลจากนักประเมิน ID=1  (แก้ไขจาก ID=01 เป็น ID=1)
estimator_data = estimator_data[estimator_data['ID'] == 1] # เปลี่ยน '01' เป็น 1  เนื่องจากคอลัมน์ ID เป็นตัวเลข

# ตรวจสอบข้อมูล
estimator_data.head()

# Select and rename relevant columns from estimator_data
prepared_data = estimator_data[['ประเมินราคาซื้อ-ขาย', 'ให้คะแนนทรัพย์']].copy() # Creating prepared_data here
prepared_data.columns = ['estimator', 'score']

# Now you can work with prepared_data
print(prepared_data.isnull().sum())  # ตรวจสอบค่า Missing
print(prepared_data.describe())  # สถิติพื้นฐาน

estimator    0
score        0
dtype: int64
          estimator
count  2.000000e+00
mean   9.500000e+05
std    4.949747e+05
min    6.000000e+05
25%    7.750000e+05
50%    9.500000e+05
75%    1.125000e+06
max    1.300000e+06


# 2 **จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)**

**1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)**
- กลุ่ม zone ที่มีราคาขายเฉลี่ย 'sell price' สูงที่สุดคือ ในเมือง-รื่นรมย์-เทพารักษ์

**2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)**
- บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่  390,000 บาท

**3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)**
- ตำบลศรีบุญเรือง	ได้คะแนนทรัพย์เฉลี่ยจากนักประเมินมากที่สุด ซึ่งเท่ากับ 4.00 คะแนน